In [1]:
from lavis.models import load_model_and_preprocess
import torch
import argparse
import time

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

print('Loading model...')
start_time = time.time()

model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_vicuna_instruct",
    model_type="vicuna7b",
    is_eval=True,
    device=device,
)
end_time = time.time()
print('Loading model done! Time cost: ', end_time - start_time, 's')

Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading model done! Time cost:  225.81879544258118 s


In [2]:
from PIL import Image

def inference2(image, prompt, num_captions):
    min_len = 15
    max_len = 95
    beam_size = 5
    len_penalty = 1
    repetition_penalty = 1.2
    top_p = 0.9
    decoding_method = "Beam search"

    use_nucleus_sampling = decoding_method == "Nucleus sampling"

    # Ensure prompts are a list of strings
    # prompts = prompts.split('\n')
    # assert len(prompts) == 4, "The number of prompts must be equal to 4."

    image = vis_processors["eval"](image).unsqueeze(0).to(device)
    samples = {
        "image": image,
        "prompt": prompt,
    }

    outputs = model.generate(
        samples,
        length_penalty=float(len_penalty),
        repetition_penalty=float(repetition_penalty),
        num_beams=beam_size,
        max_length=max_len,
        min_length=min_len,
        top_p=top_p,
        use_nucleus_sampling=use_nucleus_sampling,
        num_captions=num_captions
    )    

In [5]:
image_input = Image.open("/home/ubuntu/dev/captioning/test.jpg")
prompt = "Question: what's the image about? answer:"
inference2(image_input, prompt, 4)

/home/ubuntu/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 image_input = Image.open("/home/ubuntu/dev/captioning/test.jpg")                             │
│   2 prompt = "Question: what's the image about? answer:"                                         │
│ ❱ 3 inference2(image_input, prompt, 4)                                                           │
│   4                                                                                              │
│                                                                                                  │
│ in inference2:24                                                                                 │
│                                                                                                  │
│   21 │   │   "prompt": prompt,                                                                   │
│   22 │   }                                                                                       │
│   23 │                                                                                           │
│ ❱ 24 │   outputs = model.generate(                                                               │
│   25 │   │   samples,                                                                            │
│   26 │   │   length_penalty=float(len_penalty),                                                  │
│   27 │   │   repetition_penalty=float(repetition_penalty),                                       │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/autograd/grad_mode.py:27 in           │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/ubuntu/dev/LAVIS2/lavis/models/blip2_models/blip2_vicuna_instruct.py:373 in generate       │
│                                                                                                  │
│   370 │   │   │   )                                                                              │
│   371 │   │                                                                                      │
│   372 │   │   outputs[outputs == 0] = 2 # convert output id 0 to 2 (eos_token_id)                │
│ ❱ 373 │   │   output_text = self.llm_tokenizer.batch_decode(outputs, skip_special_tokens=True)   │
│   374 │   │   output_text = [text.strip() for text in output_text]                               │
│   375 │   │                                                                                      │
│   376 │   │   return output_text                                                                 │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/transf

In [4]:
from PIL import Image

def inference(image, prompts, num_captions):
    min_len = 15
    max_len = 95
    beam_size = 5
    len_penalty = 1
    repetition_penalty = 1.2
    top_p = 0.9
    decoding_method = "Beam search"

    use_nucleus_sampling = decoding_method == "Nucleus sampling"

    # Ensure prompts are a list of strings
    # prompts = prompts.split('\n')
    # assert len(prompts) == 4, "The number of prompts must be equal to 4."

    image = vis_processors["eval"](image).unsqueeze(0).to(device)
    images = image.repeat(len(prompts), 1, 1, 1)

    samples = {
        "image": images,
        "prompt": prompts,
    }

    outputs = model.generate(
        samples,
        length_penalty=float(len_penalty),
        repetition_penalty=float(repetition_penalty),
        num_beams=beam_size,
        max_length=max_len,
        min_length=min_len,
        top_p=top_p,
        use_nucleus_sampling=use_nucleus_sampling,
        num_captions=num_captions
    )

In [ ]:
image_input = Image.open("/home/ubuntu/dev/captioning/test.jpg")
prompts = ["what's in the image?", "Question: what's the image about? answer:", "what's the story in the image?", "what's the style of the image?"]

inference(image_input, prompts, 4)

In [7]:
# model.generate | model: 
print(type(model))

<class 'lavis.models.blip2_models.blip2_vicuna_instruct.Blip2VicunaInstruct'>


In [ ]:
image_input = Image.open("/home/ubuntu/dev/captioning/test.jpg")
print(type(image_input))
print(image_input.size)

prompts = ["what's in the image?", "Question: what's the image about? answer:"]
samples = {
    "image": image_input,
    "prompt": prompts,
}
    
model.generate(samples)

In [15]:
# Assuming you have a list of image paths and prompts
image_paths = ["/home/ubuntu/dev/captioning/test.jpg", "/home/ubuntu/dev/captioning/test.jpg"]
prompts = ["what's in the image?", "Question: what's the image about? answer:"]

images = []
for img_path in image_paths:
    image_input = Image.open(img_path)
    image_input = vis_processors["eval"](image_input).unsqueeze(0).to(device)
    images.append(image_input)

images = torch.cat(images, dim=0)  # Concatenate along the batch dimension

samples = {
    "image": images,
    "prompt": prompts,
}

output_texts = model.generate(samples)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 │   "prompt": prompts,                                                                      │
│   16 }                                                                                           │
│   17                                                                                             │
│ ❱ 18 output_texts = model.generate(samples)                                                      │
│   19                                                                                             │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/autograd/grad_mode.py:27 in           │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/ubuntu/dev/LAVIS/lavis/models/blip2_models/blip2_vicuna_instruct.py:323 in generate        │
│                                                                                                  │
│   320 │   │   │   atts_llm = torch.cat(atts_llm, dim=1)                                          │
│   321 │   │   else:                                                                              │
│   322 │   │   │   with self.maybe_autocast():                                                    │
│ ❱ 323 │   │   │   │   image_embeds = self.ln_vision(self.visual_encoder(image))                  │
│   324 │   │   │   image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image   │
│   325 │   │   │                                                                                  │
│   326 │   │   │   if self.qformer_text_input:                                                    │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in          │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_